<a href="https://colab.research.google.com/github/Chia-Yin-Lee/NTHU/blob/main/dailytransaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title show your CPU or GPU details
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12375352241958884603
 xla_global_id: -1]

In [ ]:
#@title connect google drive folder

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/thesis

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/thesis


In [ ]:
#@title SREIT symbols & sectors

# Divide the SREITs with diff sectors
healthcare_symbols = ["AW9U.SI", "C2PU.SI"]
hospitality_symbols = ["HMN.SI", "J85.SI", "Q5T.SI", "ACV.SI"]
retail_symbols = ["BMGU.SI", "C38U.SI", "AU8U.SI", "CEDU.SI", "J69U.SI", "D5IU.SI", "CRPU.SI", "SK6U.SI", "P40U.SI"]
office_symbols = ["UD1U.SI", "K71U.SI", "JYEU.SI", "N2IU.SI", "TS0U.SI", "OXMU.SI", "T82U.SI"]
industrial_symbols = ["O5RU.SI", "A17U.SI", "DHLU.SI", "BWCU.SI", "J91U.SI", "BUOU.SI", "AJBU.SI", "ME8U.SI", "M44U.SI", "M1GU.SI"]
# seit_symbols = healthcare_symbols + hospitality_symbols + retail_symbols + office_symbols + industrial_symbols

# Define the date range for the data
start_date = "2021-04-26"  # to prevent some technical indicators being NaN
end_date = "2023-08-31"

In [ ]:
#@title import packages

import yfinance as yf
import pandas as pd
import numpy as np

#import nlpaug.augmenter.char as nac
#import nlpaug.augmenter.word as naw
#import nlpaug.augmenter.sentence as nas
#import nlpaug.flow as nafc
#from gensim.models import KeyedVectors
import copy
import datetime as dt
from datetime import datetime
import string
import os
import codecs
import io
from lxml import etree
from dateutil.relativedelta import *
import seaborn as sns; sns.set(style="darkgrid")
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import re
import pickle
from tqdm.notebook import tqdm
import nltk
#import transformers

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.semi_supervised import LabelPropagation
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score

#from sklearn.feature_extraction import DictVectorizer
#from sklearn.pipeline import FeatureUnion, Pipeline
#from sklearn.compose import ColumnTransformer

Then, we'd load Daily transaction data, including

1.   Open/close price
2.   High/low price
3.   Volume
4.   VolumeChangeAmount/Rate

In [ ]:
#@title generate_dataset function (deprecated)

def generate_dataset(sector_name, start_date, end_date):

  df = pd.DataFrame()

  for symbol in sector_name:
      sreit = yf.Ticker(symbol)
      data = sreit.history(start=start_date, end=end_date)
      data['Symbol'] = symbol

      df = pd.concat([df, data])

  df = df[['Symbol', 'Open', 'Close', 'Low', 'High', 'Volume']]

  df['Change Amount'] = df['Close'] - df['Open']
  df['Change Rate'] = (df['Close'] - df['Open']) / df['Open'] * 100

  return df

In [ ]:
#@title generate_dataset function

def generate_dataset(sector_name, start_date, end_date):

  df = pd.DataFrame()

  for symbol in sector_name:
      sreit = yf.Ticker(symbol)
      data = sreit.history(start=start_date, end=end_date)
      data['Symbol'] = symbol
      df = pd.concat([df, data])

  df = df[['Symbol', 'Open', 'Close', 'Low', 'High', 'Volume']]

  # Calculate volume change amount and volume change rate
  df["VolumeChangeAmount"] = df.groupby("Symbol")["Volume"].diff()
  df["VolumeChangeRate"] = df.groupby("Symbol")["Volume"].pct_change() * 100

  # Replace NaN values with 0
  df["VolumeChangeAmount"].fillna(0, inplace=True)
  df["VolumeChangeRate"].fillna(0, inplace=True)

  new_index_name = 'Date'
  df = df.reset_index()
  df = df.rename(columns={'index': new_index_name})
  df[new_index_name] = df[new_index_name].dt.strftime('%Y-%m-%d')

  return df

In [ ]:
#@title Create an empty list to store the datasets
datasets = []

for sector in ['healthcare', 'hospitality', 'retail', 'office', 'industrial']:
    dataset_name = f'{sector}_df'
    sector_name = eval(sector + '_symbols')
    df = generate_dataset(sector_name, start_date, end_date)
    datasets.append((dataset_name, df))

for dataset_name, df in datasets:
    print(f'Dataset Name: {dataset_name}')
    print(df)
    print('\n')

Dataset Name: healthcare_df
            Date   Symbol      Open     Close      Low      High   Volume  \
0     2021-04-26  AW9U.SI  0.204240  0.208168  0.20424  0.208168  1937300   
1     2021-04-27  AW9U.SI  0.208168  0.208168  0.20424  0.212096  6043800   
2     2021-04-28  AW9U.SI  0.208168  0.208168  0.20424  0.212096  1937800   
3     2021-04-29  AW9U.SI  0.208168  0.208168  0.20424  0.212096  5080800   
4     2021-04-30  AW9U.SI  0.204240  0.204240  0.20424  0.208168  1196100   
...          ...      ...       ...       ...      ...       ...      ...   
1175  2023-08-24  C2PU.SI  3.800000  3.750000  3.75000  3.800000   244100   
1176  2023-08-25  C2PU.SI  3.740000  3.790000  3.73000  3.800000   238600   
1177  2023-08-28  C2PU.SI  3.800000  3.800000  3.77000  3.830000   188200   
1178  2023-08-29  C2PU.SI  3.800000  3.840000  3.77000  3.850000   366900   
1179  2023-08-30  C2PU.SI  3.830000  3.860000  3.83000  3.860000   324600   

      VolumeChangeAmount  VolumeChangeRate  
0 

Another set of inputs is 12 widely used technical indicators of each index, calculated based on stock trading data:

Financial technical indicators refer to indicators calculated based on stock trading data, including CCI (commodity channel index), ATR (average true range), SMI (stochastic momentum index), etc. These indicators can often reflect some regular characteristics of stock movements.

1.   **MACD**: (Moving average convergence divergence) displays trend following characteristics and momentum characteristics
2.   **CCI**: (Commodity channel index) helps to find the start and the end of a trend
3.   **ATR**: (Average true range) measures the volatility of price
4.   **BOLL**: (Bollinger Band) provides a relative definition of high and low, which aids in rigorous pattern recognition
5.   **EMA20**: 20 day Exponential Moving Average
6.   **MA5/MA10**: 5/10 day Moving Average
7.   **V_MA5/V_MA10**: 5/10 days trading volume average
8.   **MTM6/MTM12**: (6/12 month Momentum) helps pinpoint the end of a decline or advance
9.   **ROC**: (Price rate of change) shows the speed at which a stock’s price is changing
10.  **SMI**: (Stochastic Momentum Index) shows where the close price is relative to the midpoint of the same range
11.  **WVAD**: Williams's Variable Accumulation/Distribution: measures the buying and selling pressure.

In [ ]:
#@title derive_tech_indicator function
def derive_tech_indicator(old_df):

  df = copy.deepcopy(old_df)

  # Moving Average Convergence Divergence (MACD)
  # df['12-day EMA'] = df['Close'].ewm(span=12).mean()
  # df['26-day EMA'] = df['Close'].ewm(span=26).mean()
  df['MACD'] = df['Close'].ewm(span=12).mean() - df['Close'].ewm(span=26).mean()
  # df['Signal Line'] = df['MACD'].ewm(span=9).mean()

  # Commodity Channel Index (CCI)
  window = 20
  typical_price = (df['High'] + df['Low'] + df['Close']) / 3
  mean_deviation = abs(typical_price - typical_price.rolling(window=window).mean())
  cci = (typical_price - typical_price.rolling(window=window).mean()) / (0.015 * mean_deviation)
  df['CCI'] = cci

  # Average True Range (ATR)
  window = 14
  high_low = df['High'] - df['Low']
  high_close = abs(df['High'] - df['Close'].shift())
  low_close = abs(df['Low'] - df['Close'].shift())
  true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
  df['ATR'] = true_range.rolling(window=window).mean()

  # Bollinger Bands
  window = 20
  # df['20-day SMA'] = df['Close'].rolling(window=window).mean()
  df['BOLL_U'] = df['Close'].rolling(window=window).mean() + 2 * df['Close'].rolling(window=window).std()
  df['BOLL_L'] = df['Close'].rolling(window=window).mean() - 2 * df['Close'].rolling(window=window).std()

  # Exponential Moving Averages (EMA)
  df['EMA20'] = df['Close'].ewm(span=20).mean()  # 20 days exponential moving average
  df['MA5'] = df['Close'].ewm(span=5).mean()  # 5 days moving average
  df['MA10'] = df['Close'].ewm(span=10).mean()  # 10 days moving average

  # Trading Volume Averages
  df['V_MA5'] = df['Volume'].rolling(window=5).mean()
  df['V_MA10'] = df['Volume'].rolling(window=10).mean()

  # Momentum
  df['MTM6'] = df['Close'] / df['Close'].shift(126) - 1
  df['MTM12'] = df['Close'] / df['Close'].shift(252) - 1

  # Price Rate of Change (ROC)
  window = 12
  df['ROC'] = df['Close'].pct_change(periods=window) * 100

  # Stochastic Momentum Index (SMI)
  High_SMI = df['High'].rolling(window=14).max()
  Low_SMI = df['Low'].rolling(window=14).min()
  K = ((df['Close'] - Low_SMI) / (High_SMI - Low_SMI)) * 100
  D = K.rolling(window=3).mean()
  df['SMI'] = K - D

  # Williams's Variable Accumulation/Distribution (WAD)
  df['WVAD'] = (df['Close'] - df['Low'].shift() - (df['High'].shift() - df['Close'])) / (df['High'].shift() - df['Low'].shift()) * df['Volume']

  return df

In [ ]:
#@title Create Datasets with Technical Indicator and Save as .csv files

for dataset_name, df in datasets:
  new_df = derive_tech_indicator(df)

  file_path = dataset_name + '.csv'
  new_df.to_csv(file_path, index=False)

The final set of inputs is the macroeconomic variable. We select 3 kinds of macro variables to capture the impact from the monetary market to the stock market:

1.    **FX**: USD/SGD exchange rate
2.    **DFF**: Federal Funds Rate
3.    **SIBOR**: Singapore Interbank Offered Rate
4.    **VIX**: CBOE Volatility Index

In [ ]:
start_date = "2022-04-26"  # change to the real start date

SIBOR_data = pd.read_csv("SIBOR.csv")
SIBOR_data["SIBOR DATE"] = pd.to_datetime(SIBOR_data["SIBOR DATE"])
SIBOR_data = SIBOR_data.rename(columns = {"SIBOR DATE": "Date", "SIBOR 1M": "SIBOR_1M", "SIBOR 3M": "SIBOR_3M"})
SIBOR_data = SIBOR_data.set_index("Date")
#data=pd.merge(ER,SIBOR , left_on='Date', right_on='SIBOR DATE',how='inner')
SIBOR_data.head()

,SIBOR_1M,SIBOR_3M
Date,,
2022-04-26,0.67214,1.05606
2022-04-27,0.67500,1.05624
2022-04-28,0.67518,1.05679
2022-04-29,0.78554,1.09375
2022-05-04,0.78691,1.11250


In [ ]:
usdsgd_ticker = "USDSGD=X"  # Exchange Rate
fed_funds_rate_ticker = "^IRX"  # Federal Funds Rate
vix_ticker = "^VIX"  # VIX

FX_data = yf.download(usdsgd_ticker, start=start_date, end=end_date)
DFF_data = yf.download(fed_funds_rate_ticker, start=start_date, end=end_date)
VIX_data = yf.download(vix_ticker, start=start_date, end=end_date)

# Display the data
print("USD/SGD Exchange Rate:")
print(FX_data.tail())

print("\nFederal Funds Rate:")
print(DFF_data.tail())

print("\nVIX Index:")
print(VIX_data.tail())

# Combine the Macroeconomic Variable data by date
macroeconomic_data = pd.concat([FX_data['Adj Close'], DFF_data['Adj Close'], VIX_data['Adj Close'], SIBOR_data], axis=1)

# Rename the columns for clarity
macroeconomic_data.columns = ["FX", "DFF", "VIX", "SIBOR_1M", "SIBOR_3M"]

print(macroeconomic_data.tail())

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
USD/SGD Exchange Rate:
               Open     High      Low    Close  Adj Close  Volume
Date                                                             
2023-08-24  1.35180  1.35558  1.35090  1.35180    1.35180       0
2023-08-25  1.35680  1.35862  1.35291  1.35680    1.35680       0
2023-08-28  1.35544  1.35771  1.35310  1.35544    1.35544       0
2023-08-29  1.35514  1.35741  1.35069  1.35514    1.35514       0
2023-08-30  1.34950  1.35408  1.34833  1.34950    1.34950       0

Federal Funds Rate:
             Open   High    Low  Close  Adj Close  Volume
Date                                                     
2023-08-24  5.295  5.303  5.290  5.303      5.303       0
2023-08-25  5.298  5.315  5.298  5.315      5.315       0
2023-08-28  5.320  5.325  5.318  5.323      5.323      

In [ ]:
#@title Fill NaN with the value from the last day

macroeconomic_data.ffill(inplace=True)
macroeconomic_data = macroeconomic_data.reset_index()
macroeconomic_data = macroeconomic_data.rename(columns={'index': 'Date'})
macroeconomic_data['Date'] = macroeconomic_data['Date'].dt.strftime('%Y-%m-%d')

print(macroeconomic_data.tail())

           Date       FX    DFF    VIX  SIBOR_1M  SIBOR_3M
348  2023-08-25  1.35680  5.315  15.68     4.025   4.05644
349  2023-08-28  1.35544  5.323  15.08     4.025   4.05644
350  2023-08-29  1.35514  5.320  14.45     4.025   4.05644
351  2023-08-30  1.34950  5.315  13.88     4.025   4.05644
352  2023-08-31  1.34950  5.315  13.88     4.025   4.05644


In [ ]:
macroeconomic_data.to_csv('macroeconomic_df.csv', index=False)